# Image Style Transferの実装

--------------------------------
Gatys, Ecker, and Bethge(2016)の画像スタイル変換を実装する。

【参考文献】  
L. A. Gatys, A. S. Ecker, and M. Bethge,  
Image style transfer using convolutional neural networks,  
In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, pages, 2414-2423, 2016.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
CONTENT_FILE = '/home/ishiyama/image_style_transfer/image/input/test_input_01.JPG'
STYLE_FILE = '/home/ishiyama/image_style_transfer/image/style/test_style_01.jpg'

In [32]:
def read_images_as_jpeg(file):
    
    """
    JPEG Image Reader
    
    This function reads the content and style images as JPEG format.
    These image data must be square for now, different height and
    width will be able to supplied for future.
    
    Args:
        file : str. A path of the image file.
    
    Returns:
        A tuple. Each Elements are Tensor object of the read images.
    """
    
    filename_queue = tf.train.string_input_producer([file])
    reader = tf.WholeFileReader()
    key, value = reader.read(filename_queue)
    images = tf.image.decode_jpeg(value)

    # Convert read image data to RGB representation.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    tf.train.start_queue_runners(sess)
    image_rgb = sess.run(image)

    # Reshaping numpy.array to same format of 中井(2016) [CNN-03] line3 and 4(pp.205).
    height, width, channels = image_rgb.shape
    tmp = tf.placeholder(tf.float32, [None, height * width])
    x = tf.reshape(tmp, [-1, height, width, channels])

    return x

In [35]:
image = read_images_as_jpeg(
    content_file=CONTENT_FILE,
    style_file=STYLE_FILE
)

In [36]:
image

<tf.Tensor 'Reshape_3:0' shape=(?, 1200, 1600, 3) dtype=float32>

# VGGを実装する
------------------------
画像の特徴量を抽出するアルゴリズムにはSimonyan and Zisserman(2015)で提案されたCNN(VGG19)の畳込み層とプーリング層が使われている。  
ここでは、「TensorFlowで学ぶディープラーニング入門」の多層CNNの実装を参考にVGG19を構築する。  

【参考文献】  
K. Simonyan and A. Zisserman, Very Deep Convolutional Networks For Large-Scale Image Recognition, arXiv: 1409.1556v6, 2015  
中井悦司, TensorFlowで学ぶディープラーニング入門〜畳み込みニューラルネットワーク徹底解説, マイナビ出版, 2016

### 畳み込み層を実装する

In [45]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


def calculate_convolutional_layer(x, filter_conf, output_channels):

    """
    Executeing a convolutional layer task.
    
    Args:
        x               : An image data.
        filter_conf     : A dict, or a dict like. the configulations of convolution filter.
                          This must have 4 keys ("height", "width", "channels" and "num").
        output_channels : A number of channels which is output of this function.

    Returns:
        An activation of an convolutional layer.
    """

    if len(filter_conf) != 4:
        raise ValueError('filter_conf does not have enough elements.')

    if not isinstance(filter_conf, dict):
        raise TypeError('filter_conf must be the dict type, or a dict like.')

    if (('height' not in filter_conf)
        or ('width' not in filter_conf)
        or ('channels' not in filter_conf)
        or ('num' not in filter_conf)):
        raise ValueError('filter_conf must have "height", "width", "channels" and "num".\n'
                         'Please specify these 4 conditions.')
    
    W = tf.Variable(
        tf.truncated_normal(
            [filter_conf['height'],
             filter_conf['width'],
             filter_conf['channels'],
             filter_conf['num']],
            stddev=0.1
        )
    )
    h = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    b = tf.Variable(tf.constant(0.1, shape=output_channels))
    convoluted_image = tf.nn.relu(h + b)
    
    return convoluted_image

In [47]:
calculate_convolutional_layer(
    x=image,
    filter_conf={
        'height': 3,
        'width': 3,
        'channels': 1,
        'num': 1
    },
    output_channels=3
)

ValueError: Dimensions must be equal, but are 3 and 1 for 'Conv2D_9' (op: 'Conv2D') with input shapes: [?,1200,1600,3], [3,3,1,1].

### Maxプーリング層を実装する

In [9]:
def calculate_max_pooling_layer(x, ksize, strides):

    """Wrapper function of tf.nn.max_pool.
    
    Args:
        x       : A Tensor produced by calculate_convolutional_layer.
        ksize   : A list of ints that has length >= 4. The size of
                  the window for each dimension of the input tensor.
        strides : A list of ints that has length >= 4. The stride
                  of the sliding window for each dimension of the
                  input tensor.
    
    Returns:
        A pooled image.
    """

    pooled_image = tf.nn.max_pool(x, ksize=ksize, strides=strides, padding='SAME')

    return pooled_image

### 畳込みとプーリング処理の途中経過を保持するクラスを実装する

In [15]:
FILTER_CONF = {
    'height': 3,
    'width': 3,
    'channels': 1,
    'num': 1
}

class ConvNetProgressHolder(object):

    """Holder of convoluted images and pooled image.
    
    This class is used like the struct of C language.
    This has no methods.
    
    Attributes:
        input_data (Tensor) : An image that is applied to convolution and pooling.
        conv (list)         : The list of convoluted images, each images are Tensor objects.
        pool (Tensor)       : A image that is pooled after convolutional layer.
    """

    def __init__(self):

        self.input_data = None
        self.conv = []
        self.pool = None

    
def apply_vgg_network_unit(x, channels, num_conv):

    """Apply VGG Network From a Convolutional Layer to Max Pooling Layer.

    Table 1 of Simonyan and Zisserman(2015) is separated by 5 parts,
    each parts is from an input data or a pooled data at previous part
    to a maxpool.
    This function provides to apply a that part.
    This will apply recursively.
    
    Args:
        x (Tensor)     : An input data or A Max pooled data returned by this function.
        channels (int) : A number of channels described at Table 1 of
                         Simonyan and Zisserman(2015).
        num_conv (int) : A number of applying covolutional layers.
                         See Simonyan and Zisserman(2015) for detail.

    Returns:
        A ConvNetProgressHolder object.
    """

    if num_conv < 2:
        raise ValueError('num_conv must be >= 2.')

    conv_holder = ConvNetProgressHolder()
    conv_holder.input_data = x

    conv = calculate_convolutional_layer(
        x=conv_holder.input_data,
        filter_conf=FILTER_CONF,
        output_channels=channels
    )
    conv_holder.conv.append(conv)

    for i in range(1, num_conv):
        conv = calculate_convolutional_layer(
            x=conv_holder.conv[i - 1],
            filter_conf=FILTER_CONF,
            output_channels=channels
        )
        conv_holder.conv.append(conv)

    conv_holder.pool = calculate_max_pooling_layer(
        x=conv_holder.conv[i - 1],
        ksize=[1, 1, 1, 1],
        strides=[1, 1, 1, 1]
    )

    return conv_holder


### VGGの畳込みとプーリング層を構築する

In [ ]:
unit1 = apply_vgg_network_unit(x=x, channels=64, num_conv=2)
unit2 = apply_vgg_network_unit(x=unit1.pool, channels=128, num_conv=2)
unit3 = apply_vgg_network_unit(x=unit2.pool, channels=256, num_conv=4)
unit4 = apply_vgg_network_unit(x=unit3.pool, channels=512, num_conv=4)
unit5 = apply_vgg_network_unit(x=unit4.pool, channels=512, num_conv=4)